In [43]:
# Initialize PaddleOCR instance
from sklearn.cluster import KMeans
import json
import numpy as np
from scipy.spatial import distance_matrix
data = json.load(open('output/img_res.json', 'r', encoding='utf-8'))


    


In [14]:
km = KMeans(n_clusters=4, random_state=0)
x_left = [poly[0][0] for poly in data['rec_polys']]
y_left = [poly[0][1] for poly in data['rec_polys']]
x_left = np.array(x_left).reshape(-1, 1)
km.fit_predict(x_left)



array([0, 2, 3, 1, 0, 0, 3, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 2, 0, 1, 2, 3,
       0, 2, 3, 1, 0, 1, 3, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
       2, 3, 0, 1, 2, 3, 0, 2, 3, 1, 0, 1, 2, 3, 3, 0, 1, 2, 3, 0, 1, 2,
       3, 0, 1, 2, 3, 0, 1, 2, 3, 3, 0, 1, 2, 2, 3, 1, 3, 0, 1, 2, 3, 0,
       1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 2, 3, 0, 1, 3, 0, 1, 2, 0, 1, 2, 3,
       3, 0, 1, 2, 2, 1, 0, 2, 3, 2, 3, 0, 1, 1, 2, 3, 0, 1, 2, 3, 0, 3,
       0, 1, 2, 3, 0, 1, 1, 2, 3, 0, 1, 3, 0, 1, 2, 3, 0, 2, 3, 1, 0, 1,
       2, 3, 0, 1, 2, 3, 1, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2,
       0, 1, 1, 2, 3, 0, 1, 1, 2, 3, 0, 1, 1, 1, 2, 0, 1, 3, 1, 2, 0, 1,
       2, 0, 1, 1, 2, 3, 0, 1, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
       0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0],
      dtype=int32)

In [13]:
x_left

array([[1773],
       [ 712],
       [1327],
       [ 270],
       [1776],
       [2410],
       [1327],
       [ 272],
       [ 714],
       [1328],
       [1773],
       [ 272],
       [ 715],
       [1266],
       [1775],
       [ 273],
       [ 566],
       [ 713],
       [1777],
       [ 268],
       [ 715],
       [1329],
       [1779],
       [ 713],
       [1328],
       [ 269],
       [1776],
       [ 267],
       [1328],
       [ 715],
       [1778],
       [ 268],
       [ 715],
       [1328],
       [1779],
       [ 267],
       [ 715],
       [1330],
       [1778],
       [ 266],
       [ 712],
       [1330],
       [1780],
       [ 267],
       [ 715],
       [1328],
       [1780],
       [ 267],
       [ 714],
       [1329],
       [1781],
       [ 714],
       [1333],
       [ 266],
       [1782],
       [ 267],
       [ 715],
       [1333],
       [1335],
       [1783],
       [ 200],
       [ 714],
       [1332],
       [1783],
       [ 265],
       [ 715],
       [13

In [ ]:
text = [[] for _ in range(4)]
boxes = [[] for _ in range(4)]
for idx, j in enumerate(km.labels_):
        text[j].append(data['rec_texts'][idx])
        boxes[j].append(data['rec_polys'][idx])
# order clusters left to right
cluster_centers = km.cluster_centers_.flatten() 
sorted_indices = np.argsort(cluster_centers)
text = [text[i] for i in sorted_indices]
boxes = [boxes[i] for i in sorted_indices]

#sort text and boxed top to bottom within each cluster 
for i in range(4):
    y_coords = [box[0][1] for box in boxes[i]]
    sorted_indices = np.argsort(y_coords)
    text[i] = [text[i][j] for j in sorted_indices]
    boxes[i] = [boxes[i][j] for j in sorted_indices]

# remove lection numbers 
import re
lection_number_pattern = re.compile(r'\d.')
for i in range(4):
    for j in range(len(text[i])):
        text[i][j] = re.sub(lection_number_pattern, '', text[i][j]).strip() 

# match text by min vertical distance between their boxes
box_0_centers = [b[0][1] + b[0][3] / 2 for b in boxes[0]]
box_1_centers = [b[0][1] + b[0][3] / 2 for b in boxes[1]]


            


In [ ]:
results = []

for b0, it in zip(box_0_centers, text[0]):
    for b1, de in zip(box_1_centers, text[1]):
        if np.abs(b0 - b1) < 10:
            results.append(f"{it}; {de}")
            break
        if b0 < b1:
            

In [38]:
[b for b in text[0]]

['voi',
 'chi?',
 'siete inf essere*',
 'siamo infessere',
 'sei inf essere*',
 'no',
 'lui',
 'il signor...',
 'il signore',
 'sì',
 'piacere',
 'sto bene',
 'a',
 'Milano',
 'leggete inf leggere*',
 'il dialogo pl-ghi',
 'Vero?',
 'no?',
 "non c'e male",
 'non',
 'abbastanza',
 'insomma',
 'tocca a noi',
 'essere',
 'lei',
 'noi',
 'loro',
 'sono inf essere',
 'di',
 'Roma',
 'famoso/a pl-i-e',
 'lo/la studente/-essa pl',
 'studenti/studentesse',
 'il/la professore/-essa',
 'Firenze',
 'la domanda pl domande Frage',
 'la risposta pl risposte',
 'Pronti?',
 'Via!',
 'la musica',
 '',
 'il vulcano',
 "il giro d'Italia",
 'la laguna',
 'gli spaghetti pl',
 'fungo pl funghi',
 'ilt',
 '',
 'ripetete inf ripetere',
 'cioccolato',
 '',
 'il',
 'lago di Garda',
 'il',
 'il latte macchiato',
 'il',
 'cappuccino',
 'gondola',
 'lag',
 'la pausa caffe',
 'la pausa',
 'lo scioglilingua',
 'Genova',
 'il gin',
 'per',
 'Cin cin!',
 'alla stracciatella']

In [39]:

[b for b in text[1]]

['ihr; Sie',
 'wer?',
 'ihr seid; Sie sind',
 'wir sind',
 '。',
 'du bist',
 'nein',
 'er',
 'Herr ...',
 'Herr',
 'ja',
 'freut mich',
 'es geht mir gut',
 'in',
 'Mailand',
 'lesen Sie',
 'Dialog',
 'nicht wahr?',
 'oder?',
 'nicht schlecht',
 'nicht',
 'ganz, ziemlich',
 'na ja',
 'wir sind dran',
 'sein',
 'sie ( Person Singular)',
 'wir',
 'sie ( Person Plural)',
 'sie sind',
 '*',
 'aus',
 'Rom',
 'berühmt',
 'Student/in',
 'Professor/in',
 'Florenz',
 'Antwort',
 'Fertig?',
 'Los!',
 'Musik',
 'Vulkan',
 "Giro d'Italia",
 'Lagune',
 'Spaghetti',
 'Pilz',
 'wiederholen Sie',
 'Schokolade',
 'Gardasee',
 'Latte macchiato',
 'Cappuccino',
 'Gondel',
 'Kaffeepause',
 'Pause',
 'Zungenbrecher',
 'Genua',
 'Gin',
 'für',
 'Prost!',
 'Stracciatella-']